In [1]:
pip install pdfplumber


Note: you may need to restart the kernel to use updated packages.


Key Sections for parsing:
Metadata (name, branch, accounts)
For each account:
Start/end balance
List of transactions
daily balance

General Schema:
users -> id, name, address, etc
account -> id, user_id, account type, account number
statement -> id, account_id, start/end date, start/end balance
transaction, id, statment_id, date, value, description

In [2]:
#Utility functions
def parse_activity(pdf_text):
    chunk = pdf_text[pdf_text.find("Account Activity"):pdf_text.find("Daily Balances")]
    data = []
    r = re.compile(r'^\d\d\/\d\d\/\d\d\d\d') #regex matches lines that start dd/mm/yyyy without fully checking the date
    activity = list(filter(r.match, chunk.split('\n'))) #filter by regex
    for line in activity:
        date = line[:10]
        first_amount = line.find("$")
        description = line[11:first_amount - 1]
        value = float(line[first_amount + 1:line.find("$", first_amount + 1)].replace(",",""))
        if description not in ["DEPOSIT", "DIRECT DEP"]:
            value = -value
        if "Balance" not in description and len(description) > 0:
            data.append((date, description, value))
    return data
    
def parse_summary(pdf_text):
    chunk = pdf_text[pdf_text.find("Account Summary"):]
    split = chunk.split('\n')
    start_line = split[2]
    end_line = split[5]
    start_date = start_line[:10]
    end_date = end_line[:10]
    start_bal = start_line[start_line.find('$'):]
    end_bal = end_line[end_line.find('$'):]
    return (start_bal, end_bal, start_date, end_date)

In [3]:
#DB setup
import sqlite3
con = sqlite3.connect('statement.db')
cur = con.cursor()

cur.execute('''DROP TABLE IF EXISTS customers''')
cur.execute('''CREATE TABLE customers
               (id integer primary key, name text, address text)''')

cur.execute('''DROP TABLE IF EXISTS accounts''')
cur.execute('''CREATE TABLE accounts
               (id integer primary key, customer_id integer not null,
               acc_type text, acc_number integer,
               FOREIGN KEY (customer_id)
       REFERENCES customer (id) )''')

cur.execute('''DROP TABLE IF EXISTS statements''')
cur.execute('''CREATE TABLE statements
               (id integer primary key, account_id integer,
               bal_start real, bal_end real, date_start text, date_end text,
               foreign key(account_id) references accounts(id))''')

cur.execute('''DROP TABLE IF EXISTS transactions''')
cur.execute('''CREATE TABLE transactions
               (id integer primary key, statement_id integer, date text, value real, description text,
               foreign key (statement_id) references statements(id))''')
con.commit()
#TODO: add daily balances


In [4]:
#Main code
import pdfplumber
import re
import sqlite3


pdf_text = ''
with pdfplumber.open(r'sample_statement.pdf') as pdf:
    for p in pdf.pages:
        pdf_text = pdf_text + p.extract_text()
        
split = pdf_text.split('\n')

#Hard coded
name = split[6]
address = split[7]
account = split[19].split('-')

cur.execute('''INSERT INTO customers (name, address) VALUES (?, ?)''', (name, address))
cur.execute('''INSERT INTO accounts (customer_id, acc_type, acc_number) VALUES (?, ?, ?)'''
            , (cur.lastrowid, account[0], account[1]))

statement_data = parse_summary(pdf_text)
cur.execute('''INSERT INTO statements (account_id, bal_start, bal_end, date_start, date_end) VALUES (?, ?, ?, ?, ?)''',
           (cur.lastrowid, statement_data[0], statement_data[1], statement_data[2], statement_data[3]))

statement_id = cur.lastrowid
txn_data = parse_activity(pdf_text)
cur.executemany('''INSERT INTO transactions (statement_id, date, value, description) VALUES (?, ?, ?, ?)''',
           (list(map(lambda x: (cur.lastrowid, x[0], x[2], x[1]), txn_data)))) #run for each transaction, adding statement ID

con.commit()
cur.execute('''SELECT * FROM transactions''')
print(cur.fetchall())

con.close()

[(1, 1, '09/04/2018', -57.48, 'Signature POS Debit 09/02 MD BALTIMORE GIANT FOOD'), (2, 1, '09/04/2018', -1989.6, 'Nationstar dba Mr Cooper XXXXXX6179'), (3, 1, '09/05/2018', -42.99, 'HMS WARRANTY 8002473680 5829389'), (4, 1, '09/05/2018', -4671.42, 'SAMS CLUB MC ONLINE PMT CKF426104254POS'), (5, 1, '09/05/2018', -8212.0, 'DISCOVER BANK ETRANSFER '), (6, 1, '09/06/2018', -160.75, 'BLTMORE GAS ELEC ONLINE PMT '), (7, 1, '09/06/2018', -195.0, 'DEVONSHIRE II CO CONS CP BC5198'), (8, 1, '09/07/2018', 653.25, 'DEPOSIT'), (9, 1, '09/07/2018', -88.59, 'TARGET ONLINE PMT '), (10, 1, '09/10/2018', -180.0, 'ATM Withdrawal 09/07 MD BALTIMORE 10101'), (11, 1, '09/10/2018', -70.11, 'Signature POS Debit 09/08 MD BALTIMORE GIANT FOOD '), (12, 1, '09/10/2018', -12.98, 'L A FITNESS '), (13, 1, '09/11/2018', -116.22, 'AT&T MOBILITY ONLINE PMT '), (14, 1, '09/14/2018', 606.62, 'DEPOSIT'), (15, 1, '09/14/2018', -3033.33, 'DIRECT DEP '), (16, 1, '09/14/2018', -19.86, 'Signature POS Debit 09/13 MD BALTIMORE

In [5]:
#cur.execute('''SELECT * FROM transactions''')
#print(cur.fetchall())
                    